# 1. 폴더 정리

* 제공해드린 animals10.zip의 압축을 풀어 그 내용을 살펴보시고, 폴더 구조를 학습에 알맞도록 재구성해주세요
* 특히 각 클래스마다 약 30% 정도의 이미지를 test 폴더에 할당해주세요
* 중간중간 헷갈리다면 이것저것 확인하는 코드를 거쳐보세요 (ex. flat_test[:5]로 앞의 다섯값 확인)
* 궁금한 점이 있을 경우, 슬랙 질문 채널 활용을 적극 권장합니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Guideline for convenience ###
# 압축 풀기

%cd /content/drive/MyDrive/Colab Notebooks/[0823] CNN
!unzip -qq "/content/drive/MyDrive/Colab Notebooks/[0823] CNN/animals10.zip"


/content/drive/MyDrive/Colab Notebooks/[0823] CNN
^C


In [ ]:
# 각 클래스별 파일 개수 확인 -> 변수로 해당 개수 저장
# Tip) 클래스가 10개이므로 각 클래스마다 똑같은 내용의 코드를 계속 써야할까요? 클래스 이름을 리스트로 저장해 for문을 돌리면서 코드를 재사용하는건 어떨까요?
import os
from glob import glob
import os
import shutil
#Label과 Label 수 계산
print(dir)
Labels= os.listdir('Animals-10')
Labels_n=len(Labels)

#각 Label 별 데이터의 개수 저장
Nums=[0 for i in range(Labels_n)]
for num in range(Labels_n):
  file_list = glob(f'./Animals-10/{Labels[num]}/*')
  Nums[num]= len(file_list)
print(Nums)

<built-in function dir>
[2112, 1969, 3098, 1866, 4863, 1446, 2623, 1820, 4821, 1862]


In [ ]:
# test에 넣을 이미지 개수 계산
test_n=[0 for i in range(len(Nums))]
for num in range(len(Nums)):
  test_n[num]=round(Nums[num]*0.3)

print(test_n)

[634, 591, 929, 560, 1459, 434, 787, 546, 1446, 559]


In [ ]:
# train, test 폴더 경로 선언 & 만들기
from torchvision import transforms, datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

train_paths=[]
test_paths=[]



In [ ]:
# train, test 폴더로 이미지 split해서 알맞게 집어넣기
import random

def split(img_list, test_count, train_path, test_path,label):
  # img_list : 이미지 경로 리스트
  # test_count : test에 할당할 이미지 개수
  # train_path : train 데이터 넣을 경로
  # test_path : test 데이터 넣을 경로
  
  # 랜덤으로 test_count만큼 이미지 img_list에서 고르기
  # test 담을 이미지 리스트 저장
  test_files=[]
  for i in random.sample(img_list, test_count):
    test_files.append(i)

  # 위에서 고르지 않은 이미지들을 train 담을 이미지 리스트로 저장
  train_files = [x for x in img_list if x not in test_files]

  # 고른 이미지를 train_path, test_path폴더에 폭사
  for k in train_files:
    shutil.copy(k, train_path)
  
  for c in test_files:
    shutil.copy(c, test_path)

 




#label list를 이용하여, 각 라벨마다 test와 train 경로 만듦
for label_n in range(Labels_n):
  label=Labels[label_n]
  train_path=f'./train/{label}'
  test_path=f'./test/{label}'
  os.makedirs(train_path, exist_ok=True)
  os.makedirs(test_path, exist_ok=True)
  img_list=glob(f'./Animals-10/{label}/*')
  split(img_list , test_n[label_n],train_path,test_path,label) 

# 2. CNN 모델
수업 코드에 '모델 성능에 대한 고민!!' 부분을 참고해서 **test 이미지 기준 최소 80% 이상**의 모델이 나올 수 있도록 학습을 진행해주세요.
* 중간중간 헷갈리다면 이것저것 확인하는 코드를 거쳐보세요
* 90% 이상의 모델이 이상적이긴 합니다
* 궁금한 점이 있을 경우, 슬랙 질문 채널 활용을 적극 권장합니다.

전반적으로 수업 코드와 비슷하게 짜면 되는데, 생각해봐야 할 것은
* 모든 이미지를 내가 정의하는 하나의 모델에 넣어야함
* 그말은 input 데이터의 차원이 항상 동일해야 된다는 말
* 그런데 과연 내가 가지고 있는 이미지들의 사이즈가 모두 같을까? - 대략 얼마정도 사이즈 가지고 있는지 코드로 확인해보면 더 좋음 (shape 확인)
* 이미지 사이즈를 통일시키기 위해서는 어떻게 해야할까? - transforms의 Resize, RandomCrop 써볼까?
* 동물 이미지에 적합한 Augmentation은 무엇이 있을까?

In [7]:
# train, test 이미지를 위한 transforms function 정의
# 위에서 말한 포인트들 생각해보기 - 차원에 유의하자!
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import torchvision.transforms as transforms
from torchvision.transforms.functional import pad

'''
#shape 확인
#(169, 300, 3)
#(200, 300, 3)
#(194, 270, 3)
#(258, 300, 3)
#(300, 225, 3)

img_list=img_list=glob(f'./test/butterfly/*')[:5]
for img in img_list:
  img_=Image.open(img)
  img=np.array(img_)
  print(img.shape)
  '''
#Tesor 이미지는 (C, H, W)



transform_ = transforms.Compose([ 
    transforms.CenterCrop(250),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
]) 

In [9]:
# ImageFolder와 앞서 정의한 transforms function을 활용해 Dataset 객체 (train, test에 대해) 선언
import torch
import torchvision
import torchvision.transforms as transforms

train_data = torchvision.datasets.ImageFolder(
    root = '/content/drive/MyDrive/Colab Notebooks/[0823] CNN/train', # 바로 train 폴더 지정
    transform = transform_ # transform은 그냥 편의상 cifar에서 썼던거 그대로
)
test_data = torchvision.datasets.ImageFolder(
    root = '/content/drive/MyDrive/Colab Notebooks/[0823] CNN/test',
    transform = transform_
)
#print(train_data[0][0].shape)


In [16]:
# train, test에 대해 DataLoader 정의

trainloader= torch.utils.data.DataLoader(
    train_data,
    batch_size = 32,      # 일반적으로 batch size = 16, 32
    shuffle=True,         # train dataloader는 epoch마다 데이터 다시 섞어서 batch 만들고 학습! - 학습 효율 up
    num_workers=2
)

testloader = torch.utils.data.DataLoader(
    test_data,
    batch_size = 32,      # 일반적으로 batch size = 16, 32
    shuffle=False,        # test dataloader는 데이터 매번 섞을 이유가 없음 - 어차피 확인 용도
    num_workers=2
)



In [24]:
### Model 클래스 정의 - 차원에 유의하자!
# 10개의 클래스를 분류해야 하는 꽤나 복잡한 task - 모델 구조를 어느 정도로 복잡하게 짜볼까?
# 가장 마지막 Linear의 out_features는 얼마로 해야할까?
import torch.nn as nn
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        ##### Layer 정의 #####
        self.layer = nn.Sequential(
            # 맨처음 RGB 채널 3개이므로 가장 처음 in_channels = 3
            # img의 가장 첫 차원이 batch_size 값은 계속해서 유지
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5),             # [batch_size,3,32,32] -> [batch_size,16,28,28] -> same with? - using 16 filters
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5),            # [batch_size,16,28,28] -> [batch_size,32,24,24]
            nn.ReLU(),
            nn.Dropout(0.2), # 0.2 확률로 Dropout
            nn.MaxPool2d(kernel_size=2, stride=2),                                # [batch_size,32,24,24] -> [batch_size,32,12,12]
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2), # [batch_size,32,12,12] -> [batch_size,32,12,12] -> padding*2 주의!
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3),            # [batch_size,32,12,12] -> [batch_size,32,10,10]
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(kernel_size=2, stride=2)                                 # [batch_size,32,10,10] -> [batch_size,32,5,5]
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(32*5*5,100),                                                # [batch_size,32*5*5] -> [batch_size,100]
            nn.ReLU(),
            nn.Linear(100,10)
        )

    def forward(self, x):
        # forward pass 과정
        out = self.layer(x)
        out = out.view(out.size(0),-1)  # FC layer에 들어가기전, flatten! - view 함수 활용
                                        # 가장 첫 차원인 batch_size는 유지하고 나머지 차원들을 하나로 합치기 - [batch_size,16,5,5] -> [batch_size,16*5*5]
        out = self.fc_layer(out)
        return out

In [17]:
# Device 선언 (GPU 권장)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device
mymodel = MyModel().to(device)

In [25]:
# torchsummary로 모델 차원 어떻게 나오는지 돌려보기
#!pip install torchsummary
from torchsummary import summary

test_model = mymodel() # 모델 객체 선언
summary(test_model, (3,32,32), batch_size=32) # Recall) Tensor는 C, H, W 순서로 이미지 가짐

TypeError: ignored

In [ ]:
# 모델 객체 선언

In [14]:
# loss func, optimizer 정의
criterion = nn.CrossEntropyLoss().to(device) # criterion (loss func)도 device 위에서
optimizer = torch.optim.Adam(mymodel.parameters(), lr=0.001, weight_decay=0.0001) # 보통 Adam의 learning rate로 0.001 사용
                                                                                  # weight_decay = L2 Regularization의 lambda값 (가중치 제한 정도)

In [15]:
# train 함수 정의
with torch.no_grad():
    for i, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        print(inputs.shape)  # batch_size개의 이미지
        print(targets.shape) # batch_size개의 클래스

        outputs = mymodel(inputs)
        print(outputs.shape) # outputs는 32, 10 차원 (32개 data의 확률 10개씩)
        print(targets.shape) # targets는 32차원 (32개 data의 실제 클래스)

        loss = criterion(outputs, targets)  # -> cross entropy 식에 넣을때 이런 형태로 넣으면됨 (outputs, targets의 차원을 맞출 필요가 없음)
        print(loss.item())

        print(outputs.max(1)[1]) # 예측한 클래스
        print(targets) # 실제 클래스
        correct = (outputs.max(1)[1] == targets).sum().item() # 맞게 예측한 클래스 개수
        print(correct)
        break

torch.Size([32, 3, 250, 250])
torch.Size([32])


RuntimeError: ignored

In [ ]:
# test 함수 정의

In [ ]:
# training

In [ ]:
# loss, accuracy 추이 확인 with plt.plot